In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
pd.set_option('max_columns', 200)
pd.set_option('max_colwidth', 200)
pd.set_option('max_rows', 100)
warnings.filterwarnings('ignore')

%matplotlib inline

historical_transactions是过去3个月每个商户的所以交易用户信息，new_merchant_transactions是2个月内特定用户还没有涉及到的商户信息

# 观察数据

In [2]:
historical = pd.read_csv('data/historical_transactions.csv')

In [21]:
historical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29112361 entries, 0 to 29112360
Data columns (total 14 columns):
authorized_flag         object
card_id                 object
city_id                 int64
category_1              object
installments            int64
category_3              object
merchant_category_id    int64
merchant_id             object
month_lag               int64
purchase_amount         float64
purchase_date           object
category_2              float64
state_id                int64
subsector_id            int64
dtypes: float64(2), int64(6), object(6)
memory usage: 3.0+ GB


In [3]:
historical.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [5]:
states = []
for col in historical.columns:
    states.append((col, historical[col].nunique(), historical[col].isnull().sum()/historical.shape[0], historical[col].value_counts(normalize=True, dropna=False).values[0], historical[col].dtype))
state_df = pd.DataFrame(states, columns=['Feature', 'Unique values', 'Missing values', 'Percentage of biggest category', 'type'])
state_df.sort_values('Percentage of biggest category', ascending=False)

,Feature,Unique values,Missing values,Percentage of biggest category,type
3,category_1,2,0.000000,0.928414,object
0,authorized_flag,2,0.000000,0.913545,object
4,installments,15,0.000000,0.529388,int64
5,category_3,3,0.006120,0.529388,object
11,category_2,5,0.091125,0.521332,float64
12,state_id,25,0.000000,0.363038,int64
13,subsector_id,41,0.000000,0.192157,int64
6,merchant_category_id,327,0.000000,0.190778,int64
2,city_id,308,0.000000,0.166096,int64
8,month_lag,14,0.000000,0.132496,int64


In [66]:
historical['installments'].value_counts()

 0     15411747
 1     11677522
 2       666416
 3       538207
 4       179525
-1       178347
 6       132634
 10      118827
 5       116090
 12       55064
 8        20474
 7        10906
 9         5772
 11         830
Name: installments, dtype: int64

In [17]:
# purchase_date = pd.DataFrame(historical['purchase_date'].apply(lambda x: x.split(' ')[0]))
purchase_date = pd.DataFrame(purchase_date['purchase_date'].apply(lambda x: x.split('-')[0] + x.split('-')[1]))

In [27]:
purchase_date['month_lag'] = historical['month_lag']
purchase_date['purchase_date'][purchase_date['month_lag'] == -11].value_counts()

201703    872346
201702     78300
201701     57232
Name: purchase_date, dtype: int64

In [28]:
purchase_date['month_lag'] = historical['month_lag']
purchase_date['purchase_date'][purchase_date['month_lag'] == -12].value_counts()

201702    718186
201701     73787
Name: purchase_date, dtype: int64

In [29]:
purchase_date['purchase_date'][purchase_date['month_lag'] == -13].value_counts()

201701    689097
Name: purchase_date, dtype: int64

# 数据处理

In [2]:
historical = pd.read_csv('data/historical_transactions.csv')

## 时间转换

In [3]:
def purchase_day(x):
    big, small = x.split(' ')
    year, month, day = big.split('-')
    all_day = (int(year)*12+int(month)-24143)*30 + int(day)
    return all_day

def purchase_month(x):
    big, small = x.split(' ')
    year, month, day = big.split('-')
    return month

def purchase_second(x):
    big, small = x.split(' ')
    hour, minite, second = small.split(':')
    all_second = int(hour)*24*60 + int(minite)*60 + int(second)
    return all_second

In [4]:
historical['purchase_day'] = historical['purchase_date'].apply(purchase_day)
historical['purchase_second'] = historical['purchase_date'].apply(purchase_second)

In [ ]:
historical['purchase_month'] = historical['purchase_date'].apply(purchase_month)
historical = historical.drop('purchase_date', axis=1)

## 处理异常值

In [5]:
# 处理分期付款的异常值
historical['installments'][historical['installments'] == 999] = -1

## 同一merchant_id购买记录

用户在同一商家购买的次数

In [6]:
card_feature = historical[['card_id']]
card_feature = card_feature.drop_duplicates().reset_index(drop=True)

In [7]:
t = historical[['card_id', 'merchant_id']]
t['same_merchant_count'] = 1

In [8]:
t1 = t.groupby(['card_id', 'merchant_id']).count()

以card_id为结合——用户在同一家商家购买的次数的最大、最小、中位数、平均数

In [9]:
t2 = t1.reset_index().groupby('card_id').max()[['same_merchant_count']].reset_index()
t2 = t2.rename(columns={'same_merchant_count': 'same_merchant_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['same_merchant_count']].reset_index()
t3 = t3.rename(columns={'same_merchant_count': 'same_merchant_mean'})
t4 = t1.reset_index().groupby('card_id').min()[['same_merchant_count']].reset_index()
t4 = t4.rename(columns={'same_merchant_count': 'same_merchant_min'})
t5 = t1.reset_index().groupby('card_id').median()[['same_merchant_count']].reset_index()
t5 = t5.rename(columns={'same_merchant_count': 'same_merchant_median'})

In [10]:
t1['num_merchant'] = 1
t6 = t1.reset_index()[['card_id', 'num_merchant']].groupby('card_id').count().reset_index()

In [11]:
t['same_card_count'] = 1
t7 = t[['card_id', 'same_card_count']].groupby('card_id').count().reset_index()

In [12]:
card_feature = pd.merge(card_feature, t2, on='card_id', how='left')
card_feature = pd.merge(card_feature, t3, on='card_id', how='left')
card_feature = pd.merge(card_feature, t4, on='card_id', how='left')
card_feature = pd.merge(card_feature, t5, on='card_id', how='left')
card_feature = pd.merge(card_feature, t6, on='card_id', how='left')
card_feature = pd.merge(card_feature, t7, on='card_id', how='left')

In [13]:
card_feature['merchant_max_rate'] = card_feature['same_merchant_max']/card_feature['same_card_count']
card_feature['merchant_mean_rate'] = card_feature['same_merchant_mean']/card_feature['same_card_count']
card_feature['merchant_min_rate'] = card_feature['same_merchant_min']/card_feature['same_card_count']
card_feature['merchant_median_rate'] = card_feature['same_merchant_median']/card_feature['same_card_count']

In [14]:
card_feature.head()

,card_id,same_merchant_max,same_merchant_mean,same_merchant_min,same_merchant_median,num_merchant,same_card_count,merchant_max_rate,merchant_mean_rate,merchant_min_rate,merchant_median_rate
0,C_ID_4e6213e9bc,75,7.163636,1,2.0,55,400,0.187500,0.017909,0.002500,0.005000
1,C_ID_5037ff576e,8,1.475000,1,1.0,80,118,0.067797,0.012500,0.008475,0.008475
2,C_ID_0e171c1b48,29,3.861111,1,1.0,72,279,0.103943,0.013839,0.003584,0.003584
3,C_ID_48fb13e70f,8,1.837209,1,1.0,43,80,0.100000,0.022965,0.012500,0.012500
4,C_ID_fc8e41b9cf,48,2.975309,1,1.0,81,242,0.198347,0.012295,0.004132,0.004132


In [15]:
card_feature.to_csv('data/card_feature.csv', index=None)

## 同一城市购买记录

In [16]:
t = historical[['city_id', 'merchant_id', 'card_id']]
t['same_city_count'] = 1

In [17]:
t1 = t[['card_id', 'city_id', 'same_city_count']].groupby(['card_id', 'city_id']).count()

In [18]:
t2 = t1.reset_index().groupby('card_id').max()[['same_city_count']].reset_index()
t2 = t2.rename(columns={'same_city_count': 'same_city_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['same_city_count']].reset_index()
t3 = t3.rename(columns={'same_city_count': 'same_city_mean'})
t4 = t1.reset_index().groupby('card_id').min()[['same_city_count']].reset_index()
t4 = t4.rename(columns={'same_city_count': 'same_city_min'})
t5 = t1.reset_index().groupby('card_id').median()[['same_city_count']].reset_index()
t5 = t5.rename(columns={'same_city_count': 'same_city_median'})

In [19]:
t1['num_city'] = 1
t6 = t1.reset_index()[['card_id', 'num_city']].groupby('card_id').count().reset_index()

In [20]:
t['same_card_count'] = 1
t7 = t[['card_id', 'same_card_count']].groupby('card_id').count().reset_index()

In [21]:
city_feature = historical[['card_id']]
city_feature = city_feature.drop_duplicates().reset_index(drop=True)
city_feature = pd.merge(city_feature, t2, on='card_id', how='left')
city_feature = pd.merge(city_feature, t3, on='card_id', how='left')
city_feature = pd.merge(city_feature, t4, on='card_id', how='left')
city_feature = pd.merge(city_feature, t5, on='card_id', how='left')
city_feature = pd.merge(city_feature, t6, on='card_id', how='left')
city_feature = pd.merge(city_feature, t7, on='card_id', how='left')

In [22]:
city_feature['city_max_rate'] = city_feature['same_city_max']/city_feature['same_card_count']
city_feature['city_mean_rate'] = city_feature['same_city_mean']/city_feature['same_card_count']
city_feature['city_min_rate'] = city_feature['same_city_min']/city_feature['same_card_count']
city_feature['city_median_rate'] = city_feature['same_city_median']/city_feature['same_card_count']
city_feature = city_feature.drop('same_card_count', axis=1)

同一家店，不同的城市购买记录

In [23]:
t['city_merchant_count'] = 1
t1 = t[['card_id', 'city_id','merchant_id', 'city_merchant_count']].groupby(['card_id', 'merchant_id', 'city_id']).count()
t1['city_merchant_count'] = 1

In [ ]:
t2 = t1.reset_index()[['card_id', 'merchant_id', 'city_merchant_count']].groupby(['card_id', 'merchant_id']).count()

t3 = t2.reset_index().groupby('card_id').max()[['city_merchant_count']].reset_index()
t3 = t3.rename(columns={'city_merchant_count': 'city_merchant_max'})
t4 = t2.reset_index().groupby('card_id').mean()[['city_merchant_count']].reset_index()
t4 = t4.rename(columns={'city_merchant_count': 'city_merchant_mean'})

In [ ]:
def bigger_1(x):
    if x > 1:
        return 1
    else:
        return 0

In [ ]:
t5 = pd.DataFrame(t2['city_merchant_count'].apply(bigger_1).groupby('card_id').sum()).reset_index()
t5 = t5.rename(columns={'city_merchant_count': 'same_city_num_merchant'})

In [ ]:
city_feature = pd.merge(city_feature, t3, on='card_id', how='left')
city_feature = pd.merge(city_feature, t4, on='card_id', how='left')
city_feature = pd.merge(city_feature, t5, on='card_id', how='left')

In [ ]:
city_feature.head()

In [ ]:
city_feature.to_csv('data/city_feature.csv', index=None)

## 同一merchant_category_id购买记录

In [182]:
category_feature = historical[['card_id']]
category_feature = category_feature.drop_duplicates().reset_index(drop=True)

In [183]:
t = historical[['merchant_id','merchant_category_id', 'card_id']]
t['same_category_count'] = 1

In [184]:
t1 = t[['card_id', 'merchant_category_id', 'same_category_count']].groupby(['card_id', 'merchant_category_id']).count()

In [185]:
t2 = t1.reset_index().groupby('card_id').max()[['same_category_count']].reset_index()
t2 = t2.rename(columns={'same_category_count': 'same_category_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['same_category_count']].reset_index()
t3 = t3.rename(columns={'same_category_count': 'same_category_mean'})
t4 = t1.reset_index().groupby('card_id').min()[['same_category_count']].reset_index()
t4 = t4.rename(columns={'same_category_count': 'same_category_min'})
t5 = t1.reset_index().groupby('card_id').median()[['same_category_count']].reset_index()
t5 = t5.rename(columns={'same_category_count': 'same_category_median'})

In [186]:
t1['num_category'] = 1
t6 = t1.reset_index()[['card_id', 'num_category']].groupby('card_id').count().reset_index()

In [187]:
t['same_card_count'] = 1
t7 = t[['card_id', 'same_card_count']].groupby('card_id').count().reset_index()

In [188]:
category_feature = pd.merge(category_feature, t2, on='card_id', how='left')
category_feature = pd.merge(category_feature, t3, on='card_id', how='left')
category_feature = pd.merge(category_feature, t4, on='card_id', how='left')
category_feature = pd.merge(category_feature, t5, on='card_id', how='left')
category_feature = pd.merge(category_feature, t6, on='card_id', how='left')
category_feature = pd.merge(category_feature, t7, on='card_id', how='left')

In [189]:
category_feature['category_max_rate'] = category_feature['same_category_max']/category_feature['same_card_count']
category_feature['category_mean_rate'] = category_feature['same_category_mean']/category_feature['same_card_count']
category_feature['category_min_rate'] = category_feature['same_category_min']/category_feature['same_card_count']
category_feature['category_median_rate'] = category_feature['same_category_median']/category_feature['same_card_count']
category_feature = category_feature.drop('same_card_count', axis=1)

In [190]:
t['category_merchant_count'] = 1
t1 = t[['card_id', 'merchant_category_id','merchant_id', 'category_merchant_count']].groupby(['card_id','merchant_category_id', 'merchant_id']).count()
t1['category_merchant_count'] = 1

In [191]:
t2 = t1.reset_index().groupby(['card_id', 'merchant_category_id']).count()[['category_merchant_count']]

t3 = t2.reset_index().groupby('card_id').max()[['category_merchant_count']].reset_index()
t3 = t3.rename(columns={'category_merchant_count': 'category_merchant_max'})
t4 = t2.reset_index().groupby('card_id').mean()[['category_merchant_count']].reset_index()
t4 = t4.rename(columns={'category_merchant_count': 'category_merchant_mean'})

In [192]:
t5 = pd.DataFrame(t2['category_merchant_count'].apply(bigger_1).groupby('card_id').sum()).reset_index()
t5 = t5.rename(columns={'category_merchant_count': 'same_category_num_merchant'})

In [193]:
category_feature = pd.merge(category_feature, t3, on='card_id', how='left')
category_feature = pd.merge(category_feature, t4, on='card_id', how='left')
category_feature = pd.merge(category_feature, t5, on='card_id', how='left')

In [194]:
category_feature.head()

,card_id,same_category_max,same_category_mean,same_category_min,same_category_median,num_category,category_max_rate,category_mean_rate,category_min_rate,category_median_rate,category_merchant_max,category_merchant_mean,same_category_num_merchant
0,C_ID_4e6213e9bc,130,22.222222,1,8.0,18,0.325000,0.055556,0.002500,0.020000,16,3.222222,8
1,C_ID_5037ff576e,44,5.619048,1,1.0,21,0.372881,0.047619,0.008475,0.008475,25,3.857143,8
2,C_ID_0e171c1b48,64,7.750000,1,2.0,36,0.229391,0.027778,0.003584,0.007168,8,2.222222,15
3,C_ID_48fb13e70f,11,2.857143,1,1.0,28,0.137500,0.035714,0.012500,0.012500,5,1.571429,7
4,C_ID_fc8e41b9cf,48,6.914286,1,2.0,35,0.198347,0.028571,0.004132,0.008264,10,2.314286,13


In [195]:
category_feature.to_csv('data/category_feature.csv', index=None)

## 同一subsector_id

In [168]:
subsector_feature = historical[['card_id']]
subsector_feature = subsector_feature.drop_duplicates().reset_index(drop=True)
t = historical[['merchant_id','subsector_id', 'card_id']]
t['same_subsector_count'] = 1
t1 = t[['card_id', 'subsector_id', 'same_subsector_count']].groupby(['card_id', 'subsector_id']).count()

t2 = t1.reset_index().groupby('card_id').max()[['same_subsector_count']].reset_index()
t2 = t2.rename(columns={'same_subsector_count': 'same_subsector_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['same_subsector_count']].reset_index()
t3 = t3.rename(columns={'same_subsector_count': 'same_subsector_mean'})
t4 = t1.reset_index().groupby('card_id').min()[['same_subsector_count']].reset_index()
t4 = t4.rename(columns={'same_subsector_count': 'same_subsector_min'})
t5 = t1.reset_index().groupby('card_id').median()[['same_subsector_count']].reset_index()
t5 = t5.rename(columns={'same_subsector_count': 'same_subsector_median'})

t1['num_subsector'] = 1
t6 = t1.reset_index()[['card_id', 'num_subsector']].groupby('card_id').count().reset_index()
t['same_card_count'] = 1
t7 = t[['card_id', 'same_card_count']].groupby('card_id').count().reset_index()

In [169]:
subsector_feature = pd.merge(subsector_feature, t2, on='card_id', how='left')
subsector_feature = pd.merge(subsector_feature, t3, on='card_id', how='left')
subsector_feature = pd.merge(subsector_feature, t4, on='card_id', how='left')
subsector_feature = pd.merge(subsector_feature, t5, on='card_id', how='left')
subsector_feature = pd.merge(subsector_feature, t6, on='card_id', how='left')
subsector_feature = pd.merge(subsector_feature, t7, on='card_id', how='left')

In [170]:
subsector_feature['subsector_max_rate'] = subsector_feature['same_subsector_max']/subsector_feature['same_card_count']
subsector_feature['subsector_mean_rate'] = subsector_feature['same_subsector_mean']/subsector_feature['same_card_count']
subsector_feature['subsector_min_rate'] = subsector_feature['same_subsector_min']/subsector_feature['same_card_count']
subsector_feature['subsector_median_rate'] = subsector_feature['same_subsector_median']/subsector_feature['same_card_count']
subsector_feature = subsector_feature.drop('same_card_count', axis=1)

In [173]:
t['subsector_merchant_count'] = 1
t1 = t[['card_id', 'subsector_id','merchant_id', 'subsector_merchant_count']].groupby(['card_id','subsector_id', 'merchant_id']).count()
t1['subsector_merchant_count'] = 1

In [178]:
t2 = t1.reset_index().groupby(['card_id', 'subsector_id']).count()[['subsector_merchant_count']]

t3 = t2.reset_index().groupby('card_id').max()[['subsector_merchant_count']].reset_index()
t3 = t3.rename(columns={'subsector_merchant_count': 'subsector_merchant_max'})
t4 = t2.reset_index().groupby('card_id').mean()[['subsector_merchant_count']].reset_index()
t4 = t4.rename(columns={'subsector_merchant_count': 'subsector_merchant_mean'})
t5 = pd.DataFrame(t2['subsector_merchant_count'].apply(bigger_1).groupby('card_id').sum()).reset_index()
t5 = t5.rename(columns={'subsector_merchant_count': 'same_subsector_num_merchant'})

In [179]:
subsector_feature = pd.merge(subsector_feature, t3, on='card_id', how='left')
subsector_feature = pd.merge(subsector_feature, t4, on='card_id', how='left')
subsector_feature = pd.merge(subsector_feature, t5, on='card_id', how='left')

In [180]:
subsector_feature.head()

,card_id,same_subsector_max,same_subsector_mean,same_subsector_min,same_subsector_median,num_subsector,subsector_max_rate,subsector_mean_rate,subsector_min_rate,subsector_median_rate,subsector_merchant_max,subsector_merchant_mean,same_subsector_num_merchant
0,C_ID_4e6213e9bc,229,36.363636,3,12.0,11,0.572500,0.090909,0.007500,0.030000,27,5.181818,8
1,C_ID_5037ff576e,45,8.428571,1,2.5,14,0.381356,0.071429,0.008475,0.021186,26,5.785714,8
2,C_ID_0e171c1b48,64,16.411765,1,5.0,17,0.229391,0.058824,0.003584,0.017921,18,4.529412,13
3,C_ID_48fb13e70f,22,5.000000,1,3.5,16,0.275000,0.062500,0.012500,0.043750,10,2.750000,9
4,C_ID_fc8e41b9cf,53,12.100000,1,6.0,20,0.219008,0.050000,0.004132,0.024793,13,4.050000,13


In [181]:
subsector_feature.to_csv('data/subsector_feature.csv', index=None)

## 同一state_id购买记录

In [196]:
state_feature = historical[['card_id']]
state_feature = state_feature.drop_duplicates().reset_index(drop=True)
t = historical[['merchant_id','state_id', 'card_id']]
t['same_state_count'] = 1
t1 = t[['card_id', 'state_id', 'same_state_count']].groupby(['card_id', 'state_id']).count()

t2 = t1.reset_index().groupby('card_id').max()[['same_state_count']].reset_index()
t2 = t2.rename(columns={'same_state_count': 'same_state_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['same_state_count']].reset_index()
t3 = t3.rename(columns={'same_state_count': 'same_state_mean'})
t4 = t1.reset_index().groupby('card_id').min()[['same_state_count']].reset_index()
t4 = t4.rename(columns={'same_state_count': 'same_state_min'})
t5 = t1.reset_index().groupby('card_id').median()[['same_state_count']].reset_index()
t5 = t5.rename(columns={'same_state_count': 'same_state_median'})

t1['num_state'] = 1
t6 = t1.reset_index()[['card_id', 'num_state']].groupby('card_id').count().reset_index()
t['same_card_count'] = 1
t7 = t[['card_id', 'same_card_count']].groupby('card_id').count().reset_index()

state_feature = pd.merge(state_feature, t2, on='card_id', how='left')
state_feature = pd.merge(state_feature, t3, on='card_id', how='left')
state_feature = pd.merge(state_feature, t4, on='card_id', how='left')
state_feature = pd.merge(state_feature, t5, on='card_id', how='left')
state_feature = pd.merge(state_feature, t6, on='card_id', how='left')
state_feature = pd.merge(state_feature, t7, on='card_id', how='left')

state_feature['state_max_rate'] = state_feature['same_state_max']/state_feature['same_card_count']
state_feature['state_mean_rate'] = state_feature['same_state_mean']/state_feature['same_card_count']
state_feature['state_min_rate'] = state_feature['same_state_min']/state_feature['same_card_count']
state_feature['state_median_rate'] = state_feature['same_state_median']/state_feature['same_card_count']
state_feature = state_feature.drop('same_card_count', axis=1)

In [199]:
t['state_merchant_count'] = 1
t1 = t[['card_id', 'state_id','merchant_id', 'state_merchant_count']].groupby(['card_id','state_id', 'merchant_id']).count()
t1['state_merchant_count'] = 1

t2 = t1.reset_index().groupby(['card_id', 'state_id']).count()[['state_merchant_count']]

t3 = t2.reset_index().groupby('card_id').max()[['state_merchant_count']].reset_index()
t3 = t3.rename(columns={'state_merchant_count': 'state_merchant_max'})
t4 = t2.reset_index().groupby('card_id').mean()[['state_merchant_count']].reset_index()
t4 = t4.rename(columns={'state_merchant_count': 'state_merchant_mean'})
t5 = pd.DataFrame(t2['state_merchant_count'].apply(bigger_1).groupby('card_id').sum()).reset_index()
t5 = t5.rename(columns={'state_merchant_count': 'same_state_num_merchant'})

state_feature = pd.merge(state_feature, t3, on='card_id', how='left')
state_feature = pd.merge(state_feature, t4, on='card_id', how='left')
state_feature = pd.merge(state_feature, t5, on='card_id', how='left')

In [202]:
state_feature.head()

,card_id,same_state_max,same_state_mean,same_state_min,same_state_median,num_state,state_max_rate,state_mean_rate,state_min_rate,state_median_rate,state_merchant_max,state_merchant_mean,same_state_num_merchant
0,C_ID_4e6213e9bc,346,133.333333,16,38.0,3,0.865000,0.333333,0.040000,0.095000,37,18.333333,3
1,C_ID_5037ff576e,32,7.375000,1,5.0,16,0.271186,0.062500,0.008475,0.042373,22,5.125000,13
2,C_ID_0e171c1b48,238,93.000000,19,22.0,3,0.853047,0.333333,0.068100,0.078853,69,24.333333,2
3,C_ID_48fb13e70f,75,40.000000,5,40.0,2,0.937500,0.500000,0.062500,0.500000,39,21.500000,2
4,C_ID_fc8e41b9cf,176,48.400000,2,10.0,5,0.727273,0.200000,0.008264,0.041322,71,16.400000,5


In [203]:
state_feature.to_csv('data/state_feature.csv', index=None)

## Purchase_day特征

此处的purchase_day是一个相对的时间，在与train文件结合的时候要换算成train的相对时间

In [6]:
t = historical[['card_id', 'purchase_day', 'merchant_id']]
t1 = t[['card_id', 'purchase_day']].groupby(['card_id']).max()
t1 = t1.rename(columns={'purchase_day': 'purchase_day_max'})
t2 = t[['card_id', 'purchase_day']].groupby(['card_id']).min()
t2 = t2.rename(columns={'purchase_day': 'purchase_day_min'})
t9 = t[['card_id', 'purchase_day']].groupby(['card_id']).std()
t9 = t9.rename(columns={'purchase_day': 'purchase_day_std'})
t10 = t[['card_id', 'purchase_day']].groupby(['card_id']).var()
t10 = t10.rename(columns={'purchase_day': 'purchase_day_var'})

In [7]:
t3 = t.groupby(['card_id', 'merchant_id']).max() - t.groupby(['card_id', 'merchant_id']).min()

In [8]:
def bigger_0(x):
    if x > 0:
        return 1
    else:
        return 0

In [9]:
t4 = t3.reset_index().groupby('card_id').max()[['purchase_day']].reset_index()
t4 = t4.rename(columns={'purchase_day': 'time_diff_max'})
t5 = t3.reset_index().groupby('card_id').mean()[['purchase_day']].reset_index()
t5 = t5.rename(columns={'purchase_day': 'time_diff_mean'})
t6 = t3.reset_index().groupby('card_id').sum()[['purchase_day']].reset_index()
t6 = t6.rename(columns={'purchase_day': 'time_diff_sum'})
t7 = pd.DataFrame(t3['purchase_day'].apply(bigger_0).groupby('card_id').sum()).reset_index()
t7 = t7.rename(columns={'purchase_day': 'time_diff_bigger0'})

In [10]:
t8 = pd.DataFrame(t1['purchase_day_max'] - t2['purchase_day_min'], columns=['purchase_day_diff'])

In [11]:
day_feature = historical[['card_id']]
day_feature = day_feature.drop_duplicates().reset_index(drop=True)
day_feature = pd.merge(day_feature, t1, on='card_id', how='left')
day_feature = pd.merge(day_feature, t2, on='card_id', how='left')
day_feature = pd.merge(day_feature, t4, on='card_id', how='left')
day_feature = pd.merge(day_feature, t5, on='card_id', how='left')
day_feature = pd.merge(day_feature, t6, on='card_id', how='left')
day_feature = pd.merge(day_feature, t7, on='card_id', how='left')
day_feature = pd.merge(day_feature, t8, on='card_id', how='left')
day_feature = pd.merge(day_feature, t9, on='card_id', how='left')
day_feature = pd.merge(day_feature, t10, on='card_id', how='left')

In [12]:
day_feature.head()

,card_id,purchase_day_max,purchase_day_min,time_diff_max,time_diff_mean,time_diff_sum,time_diff_bigger0,purchase_day_diff,purchase_day_std,purchase_day_var
0,C_ID_4e6213e9bc,2278,1880,270,73.672727,4052,33,398,99.720433,9944.164787
1,C_ID_5037ff576e,2215,1866,271,26.912500,2153,14,349,104.524062,10925.279444
2,C_ID_0e171c1b48,2277,1915,350,80.430556,5791,32,362,104.118368,10840.634512
3,C_ID_48fb13e70f,2273,2011,258,39.627907,1704,14,262,87.066652,7580.601899
4,C_ID_fc8e41b9cf,2275,2051,214,38.407407,3111,28,224,68.909370,4748.501235


In [13]:
day_feature.to_csv('data/day_feature.csv', index=None)

## purchase_month 特征

In [32]:
t = historical[['card_id', 'purchase_day', 'merchant_id']]
t['purchase_all_month'] = t['purchase_day']//30
t = t.drop('purchase_day', axis=1)
t1 = t[['card_id', 'purchase_all_month']].groupby(['card_id']).max()
t1 = t1.rename(columns={'purchase_all_month': 'purchase_all_month_max'})
t2 = t[['card_id', 'purchase_all_month']].groupby(['card_id']).min()
t2 = t2.rename(columns={'purchase_all_month': 'purchase_all_month_min'})

t3 = t.groupby(['card_id', 'merchant_id']).max() - t.groupby(['card_id', 'merchant_id']).min()

t4 = t3.reset_index().groupby('card_id').max()[['purchase_all_month']].reset_index()
t4 = t4.rename(columns={'purchase_all_month': 'month_diff_max'})
t5 = t3.reset_index().groupby('card_id').mean()[['purchase_all_month']].reset_index()
t5 = t5.rename(columns={'purchase_all_month': 'month_diff_mean'})
t6 = t3.reset_index().groupby('card_id').sum()[['purchase_all_month']].reset_index()
t6 = t6.rename(columns={'purchase_all_month': 'month_diff_sum'})
t7 = pd.DataFrame(t3['purchase_all_month'].apply(bigger_0).groupby('card_id').sum()).reset_index()
t7 = t7.rename(columns={'purchase_all_month': 'month_diff_bigger0'})

In [33]:
month_feature = historical[['card_id']]
month_feature = month_feature.drop_duplicates().reset_index(drop=True)
month_feature = pd.merge(month_feature, t1, on='card_id', how='left')
month_feature = pd.merge(month_feature, t2, on='card_id', how='left')
month_feature = pd.merge(month_feature, t4, on='card_id', how='left')
month_feature = pd.merge(month_feature, t5, on='card_id', how='left')
month_feature = pd.merge(month_feature, t6, on='card_id', how='left')
month_feature = pd.merge(month_feature, t7, on='card_id', how='left')

In [34]:
month_feature.head()

,card_id,purchase_month_max,purchase_month_min,month_diff_max,month_diff_mean,month_diff_sum,month_diff_bigger0
0,C_ID_4e6213e9bc,75,62,9,2.400000,132,26
1,C_ID_5037ff576e,73,62,9,0.862500,69,14
2,C_ID_0e171c1b48,75,63,11,2.652778,191,30
3,C_ID_48fb13e70f,75,67,8,1.255814,54,10
4,C_ID_fc8e41b9cf,75,68,7,1.308642,106,26


In [35]:
month_feature.to_csv('data/month_feature.csv', index=None)

## 

## purchase_second特征（购买方式）

In [76]:
t = historical[['card_id', 'purchase_second']]
t['second_count'] = 1
t1 = t.groupby(['card_id', 'purchase_second']).count().reset_index()

t2 = t1.reset_index().groupby('card_id').max()[['second_count']].reset_index()
t2 = t2.rename(columns={'second_count': 'second_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['second_count']].reset_index()
t3 = t3.rename(columns={'second_count': 'second_mean'})
t4 = t1.reset_index().groupby('card_id').max().reset_index()
t4 = t4[['card_id', 'purchase_second']][t4['second_count'] != 1]
t4 = t4.rename(columns={'purchase_second': 'max_second_time'})

In [77]:
second_feature = historical[['card_id']]
second_feature = second_feature.drop_duplicates().reset_index(drop=True)
second_feature = pd.merge(second_feature, t2, on='card_id', how='left')
second_feature = pd.merge(second_feature, t3, on='card_id', how='left')
second_feature = pd.merge(second_feature, t4, on='card_id', how='left')

In [78]:
second_feature.head()

,card_id,second_max,second_mean,max_second_time
0,C_ID_4e6213e9bc,5,1.025641,36405.0
1,C_ID_5037ff576e,5,1.044248,35383.0
2,C_ID_0e171c1b48,2,1.007220,36202.0
3,C_ID_48fb13e70f,9,1.111111,36564.0
4,C_ID_fc8e41b9cf,12,1.070796,36340.0


In [79]:
second_feature.to_csv('data/second_feature.csv', index=None)

## installments特征

In [93]:
t = historical[['card_id', 'installments']]
t['installments_count'] = 1
t1 = t.groupby(['card_id', 'installments']).count().reset_index()

t2 = t1.reset_index().groupby('card_id').max()[['installments_count']].reset_index()
t2 = t2.rename(columns={'installments_count': 'install_num_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['installments_count']].reset_index()
t3 = t3.rename(columns={'installments_count': 'install_num_mean'})
t4 = t1.reset_index().groupby('card_id').median()[['installments_count']].reset_index()
t4 = t4.rename(columns={'installments_count': 'install_num_median'})
t5 = t1.reset_index().groupby('card_id').max().reset_index()
t5 = t5[['card_id', 'installments']][t5['installments_count'] != -1]
t5 = t5.rename(columns={'installments': 'max_installments'})
t6 = t1.reset_index().groupby('card_id').median().reset_index()
t6 = t6[['card_id', 'installments']][t6['installments_count'] != -1]
t6 = t6.rename(columns={'installments': 'median_installments'})

t7 = t.groupby('card_id').mean()[['installments']].reset_index()
t7 = t7.rename(columns={'installments': 'mean_installments_value'})
t8 = t.groupby('card_id').max()[['installments']].reset_index()
t8 = t8.rename(columns={'installments': 'max_installments_value'})
t9 = t.groupby('card_id').min()[['installments']].reset_index()
t9 = t9.rename(columns={'installments': 'min_installments_value'})
t10 = t.groupby('card_id').median()[['installments']].reset_index()
t10 = t10.rename(columns={'installments': 'median_installments_value'})

In [94]:
installments_feature = historical[['card_id']]
installments_feature = installments_feature.drop_duplicates().reset_index(drop=True)
installments_feature = pd.merge(installments_feature, t2, on='card_id', how='left')
installments_feature = pd.merge(installments_feature, t3, on='card_id', how='left')
installments_feature = pd.merge(installments_feature, t4, on='card_id', how='left')
installments_feature = pd.merge(installments_feature, t5, on='card_id', how='left')
installments_feature = pd.merge(installments_feature, t6, on='card_id', how='left')
installments_feature = pd.merge(installments_feature, t7, on='card_id', how='left')
installments_feature = pd.merge(installments_feature, t8, on='card_id', how='left')
installments_feature = pd.merge(installments_feature, t9, on='card_id', how='left')
installments_feature = pd.merge(installments_feature, t10, on='card_id', how='left')

In [95]:
installments_feature.head()

,card_id,install_num_max,install_num_mean,install_num_median,max_installments,median_installments,mean_installments_value,max_installments_value,min_installments_value,median_installments_value
0,C_ID_4e6213e9bc,400,400.000000,400.0,0,0.0,0.000000,0,0,0.0
1,C_ID_5037ff576e,116,39.333333,1.0,5,3.0,1.050847,5,1,1.0
2,C_ID_0e171c1b48,276,139.500000,139.5,1,0.5,0.010753,1,0,0.0
3,C_ID_48fb13e70f,72,20.000000,3.5,4,2.5,1.162500,4,1,1.0
4,C_ID_fc8e41b9cf,203,30.250000,6.5,10,3.5,1.376033,10,-1,1.0


In [96]:
installments_feature.to_csv('data/installments_feature.csv', index=None)

## purchase_amount特征

In [103]:
t = historical[['card_id', 'purchase_amount']]
t['purchase_count'] = 1
t1 = t.groupby(['card_id', 'purchase_amount']).count().reset_index()

t2 = t1.reset_index().groupby('card_id').max()[['purchase_count']].reset_index()
t2 = t2.rename(columns={'purchase_count': 'purchase_num_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['purchase_count']].reset_index()
t3 = t3.rename(columns={'purchase_count': 'purchase_num_mean'})
t4 = t1.reset_index().groupby('card_id').median()[['purchase_count']].reset_index()
t4 = t4.rename(columns={'purchase_count': 'purchase_num_median'})
t4 = t1.reset_index().groupby('card_id').min()[['purchase_count']].reset_index()
t4 = t4.rename(columns={'purchase_count': 'purchase_num_min'})
t5 = t1.reset_index().groupby('card_id').max().reset_index()
t5 = t5[['card_id', 'purchase_amount']][t5['purchase_count'] != -1]
t5 = t5.rename(columns={'purchase_amount': 'max_purchase'})
t6 = t1.reset_index().groupby('card_id').median().reset_index()
t6 = t6[['card_id', 'purchase_amount']][t6['purchase_count'] != -1]
t6 = t6.rename(columns={'purchase_amount': 'median_purchase'})

In [104]:
t7 = t.groupby('card_id').mean()[['purchase_amount']].reset_index()
t7 = t7.rename(columns={'purchase_amount': 'mean_purchase_amount_value'})
t8 = t.groupby('card_id').max()[['purchase_amount']].reset_index()
t8 = t8.rename(columns={'purchase_amount': 'max_purchase_amount_value'})
t9 = t.groupby('card_id').min()[['purchase_amount']].reset_index()
t9 = t9.rename(columns={'purchase_amount': 'min_purchase_amount_value'})
t10 = t.groupby('card_id').median()[['purchase_amount']].reset_index()
t10 = t10.rename(columns={'purchase_amount': 'median_purchase_amount_value'})

In [105]:
purchase_feature = historical[['card_id']]
purchase_feature = purchase_feature.drop_duplicates().reset_index(drop=True)
purchase_feature = pd.merge(purchase_feature, t2, on='card_id', how='left')
purchase_feature = pd.merge(purchase_feature, t3, on='card_id', how='left')
purchase_feature = pd.merge(purchase_feature, t4, on='card_id', how='left')
purchase_feature = pd.merge(purchase_feature, t5, on='card_id', how='left')
purchase_feature = pd.merge(purchase_feature, t6, on='card_id', how='left')
purchase_feature = pd.merge(purchase_feature, t7, on='card_id', how='left')
purchase_feature = pd.merge(purchase_feature, t8, on='card_id', how='left')
purchase_feature = pd.merge(purchase_feature, t9, on='card_id', how='left')
purchase_feature = pd.merge(purchase_feature, t10, on='card_id', how='left')

In [106]:
purchase_feature.head()

,card_id,purchase_num_max,purchase_num_mean,purchase_num_min,max_purchase,median_purchase,mean_purchase_amount_value,max_purchase_amount_value,min_purchase_amount_value,median_purchase_amount_value
0,C_ID_4e6213e9bc,38,2.197802,1,-0.249230,-0.727636,-0.726847,-0.249230,-0.745405,-0.734135
1,C_ID_5037ff576e,7,1.134615,1,1.567175,-0.634074,-0.368180,1.567175,-0.716434,-0.637830
2,C_ID_0e171c1b48,12,2.007194,1,-0.296112,-0.723391,-0.707345,-0.296112,-0.745405,-0.728125
3,C_ID_48fb13e70f,3,1.142857,1,0.764910,-0.635809,-0.566132,0.764910,-0.737892,-0.641031
4,C_ID_fc8e41b9cf,7,1.337017,1,3.240738,-0.619814,-0.455373,3.240738,-0.731881,-0.628198


In [107]:
purchase_feature.to_csv('data/purchase_feature.csv', index=None)

## purchase_installments联合特征

In [112]:
t = historical[['card_id', 'installments', 'purchase_amount']]
t1 = t[['card_id', 'purchase_amount']].groupby(['card_id']).max()[['purchase_amount']]
t1 = t1.rename(columns={'purchase_amount': 'purchase_max'})
t1['install_max'] = t[['card_id', 'installments']].groupby(['card_id']).max()[['installments']]

In [120]:
def change_not_0(y):
    if y == 0 or y == -1:
        return 1
    else:
        return y

In [124]:
t1['install_max'] = t1['install_max'].apply(change_not_0)
t1['every_max_install_amount'] = t1['purchase_max']/t1['install_max']
t1 = t1[['every_max_install_amount']]

In [126]:
t2 = t[['card_id', 'purchase_amount']].groupby(['card_id']).mean()[['purchase_amount']]
t2 = t2.rename(columns={'purchase_amount': 'purchase_mean'})
t2['install_mean'] = t[['card_id', 'installments']].groupby(['card_id']).mean()[['installments']]
t2['install_mean'] = t2['install_mean'].apply(change_not_0)
t2['every_mean_install_amount'] = t2['purchase_mean']/t2['install_mean']
t2 = t2[['every_mean_install_amount']]

In [127]:
t3 = t[['card_id', 'purchase_amount']].groupby(['card_id']).median()[['purchase_amount']]
t3 = t3.rename(columns={'purchase_amount': 'purchase_median'})
t3['install_median'] = t[['card_id', 'installments']].groupby(['card_id']).median()[['installments']]
t3['install_median'] = t3['install_median'].apply(change_not_0)
t3['every_median_install_amount'] = t3['purchase_median']/t3['install_median']
t3 = t3[['every_median_install_amount']]

In [128]:
install_purchase_feature = historical[['card_id']]
install_purchase_feature = install_purchase_feature.drop_duplicates().reset_index(drop=True)
install_purchase_feature = pd.merge(install_purchase_feature, t1, on='card_id', how='left')
install_purchase_feature = pd.merge(install_purchase_feature, t2, on='card_id', how='left')
install_purchase_feature = pd.merge(install_purchase_feature, t3, on='card_id', how='left')

In [129]:
install_purchase_feature.head()

,card_id,every_max_install_amount,every_mean_install_amount,every_median_install_amount
0,C_ID_4e6213e9bc,-0.249230,-0.726847,-0.734135
1,C_ID_5037ff576e,0.313435,-0.350365,-0.637830
2,C_ID_0e171c1b48,-0.296112,-65.783093,-0.728125
3,C_ID_48fb13e70f,0.191227,-0.486995,-0.641031
4,C_ID_fc8e41b9cf,0.324074,-0.330932,-0.628198


In [130]:
install_purchase_feature.to_csv('data/install_purchase_feature.csv', index=None)

## category特征

In [5]:
category_features = ['category_1', 'category_2', 'category_3']
for col in category_features:
    historical[col] = historical[col].map(dict(zip(historical[col].unique(), range(0, historical[col].nunique()))))

In [15]:
category_1_feature = historical[['card_id']]
category_1_feature = category_1_feature.drop_duplicates().reset_index(drop=True)
t = historical[['merchant_id','category_1', 'card_id']]
t['same_category_1_count'] = 1
t1 = t[['card_id', 'category_1', 'same_category_1_count']].groupby(['card_id', 'category_1']).count()

t2 = t1.reset_index().groupby('card_id').max()[['same_category_1_count']].reset_index()
t2 = t2.rename(columns={'same_category_1_count': 'same_category_1_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['same_category_1_count']].reset_index()
t3 = t3.rename(columns={'same_category_1_count': 'same_category_1_mean'})
t4 = t1.reset_index().groupby('card_id').min()[['same_category_1_count']].reset_index()
t4 = t4.rename(columns={'same_category_1_count': 'same_category_1_min'})
t5 = t1.reset_index().groupby('card_id').median()[['same_category_1_count']].reset_index()
t5 = t5.rename(columns={'same_category_1_count': 'same_category_1_median'})

t6 = t[['card_id', 'category_1']].groupby('card_id').mean().reset_index()
t6 = t6.rename(columns={'category_1': 'mean_category_1_value'})
t7 = t[['card_id', 'category_1']].groupby('card_id').max().reset_index()
t7 = t7.rename(columns={'category_1': 'max_category_1_value'})
t8 = t[['card_id', 'category_1']].groupby('card_id').min().reset_index()
t8 = t8.rename(columns={'category_1': 'min_category_1_value'})
t9 = t[['card_id', 'category_1']].groupby('card_id').median().reset_index()
t9 = t9.rename(columns={'category_1': 'median_category_1_value'})

In [16]:
category_1_feature = historical[['card_id']]
category_1_feature = category_1_feature.drop_duplicates().reset_index(drop=True)
category_1_feature = pd.merge(category_1_feature, t2, on='card_id', how='left')
category_1_feature = pd.merge(category_1_feature, t3, on='card_id', how='left')
category_1_feature = pd.merge(category_1_feature, t4, on='card_id', how='left')
category_1_feature = pd.merge(category_1_feature, t5, on='card_id', how='left')
category_1_feature = pd.merge(category_1_feature, t6, on='card_id', how='left')
category_1_feature = pd.merge(category_1_feature, t7, on='card_id', how='left')
category_1_feature = pd.merge(category_1_feature, t8, on='card_id', how='left')
category_1_feature = pd.merge(category_1_feature, t9, on='card_id', how='left')

In [17]:
category_1_feature.head()

,card_id,same_category_1_max,same_category_1_mean,same_category_1_min,same_category_1_median,mean_category_1_value,max_category_1_value,min_category_1_value,median_category_1_value
0,C_ID_4e6213e9bc,400,400.0,400,400.0,0.000000,0,0,0.0
1,C_ID_5037ff576e,118,118.0,118,118.0,0.000000,0,0,0.0
2,C_ID_0e171c1b48,279,279.0,279,279.0,0.000000,0,0,0.0
3,C_ID_48fb13e70f,75,40.0,5,40.0,0.062500,1,0,0.0
4,C_ID_fc8e41b9cf,241,121.0,1,121.0,0.004132,1,0,0.0


In [18]:
category_1_feature.to_csv('data/category_1_feature.csv', index=None)

In [19]:
category_2_feature = historical[['card_id']]
category_2_feature = category_2_feature.drop_duplicates().reset_index(drop=True)
t = historical[['merchant_id','category_2', 'card_id']]
t['same_category_2_count'] = 1
t1 = t[['card_id', 'category_2', 'same_category_2_count']].groupby(['card_id', 'category_2']).count()

t2 = t1.reset_index().groupby('card_id').max()[['same_category_2_count']].reset_index()
t2 = t2.rename(columns={'same_category_2_count': 'same_category_2_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['same_category_2_count']].reset_index()
t3 = t3.rename(columns={'same_category_2_count': 'same_category_2_mean'})
t4 = t1.reset_index().groupby('card_id').min()[['same_category_2_count']].reset_index()
t4 = t4.rename(columns={'same_category_2_count': 'same_category_2_min'})
t5 = t1.reset_index().groupby('card_id').median()[['same_category_2_count']].reset_index()
t5 = t5.rename(columns={'same_category_2_count': 'same_category_2_median'})

t6 = t[['card_id', 'category_2']].groupby('card_id').mean().reset_index()
t6 = t6.rename(columns={'category_2': 'mean_category_2_value'})
t7 = t[['card_id', 'category_2']].groupby('card_id').max().reset_index()
t7 = t7.rename(columns={'category_2': 'max_category_2_value'})
t8 = t[['card_id', 'category_2']].groupby('card_id').min().reset_index()
t8 = t8.rename(columns={'category_2': 'min_category_2_value'})
t9 = t[['card_id', 'category_2']].groupby('card_id').median().reset_index()
t9 = t9.rename(columns={'category_2': 'median_category_2_value'})

In [20]:
category_2_feature = historical[['card_id']]
category_2_feature = category_2_feature.drop_duplicates().reset_index(drop=True)
category_2_feature = pd.merge(category_2_feature, t2, on='card_id', how='left')
category_2_feature = pd.merge(category_2_feature, t3, on='card_id', how='left')
category_2_feature = pd.merge(category_2_feature, t4, on='card_id', how='left')
category_2_feature = pd.merge(category_2_feature, t5, on='card_id', how='left')
category_2_feature = pd.merge(category_2_feature, t6, on='card_id', how='left')
category_2_feature = pd.merge(category_2_feature, t7, on='card_id', how='left')
category_2_feature = pd.merge(category_2_feature, t8, on='card_id', how='left')
category_2_feature = pd.merge(category_2_feature, t9, on='card_id', how='left')

In [21]:
category_2_feature.to_csv('data/category_2_feature.csv', index=None)

In [22]:
t = historical[['merchant_id','category_3', 'card_id']]
t['same_category_3_count'] = 1
t1 = t[['card_id', 'category_3', 'same_category_3_count']].groupby(['card_id', 'category_3']).count()

t2 = t1.reset_index().groupby('card_id').max()[['same_category_3_count']].reset_index()
t2 = t2.rename(columns={'same_category_3_count': 'same_category_3_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['same_category_3_count']].reset_index()
t3 = t3.rename(columns={'same_category_3_count': 'same_category_3_mean'})
t4 = t1.reset_index().groupby('card_id').min()[['same_category_3_count']].reset_index()
t4 = t4.rename(columns={'same_category_3_count': 'same_category_3_min'})
t5 = t1.reset_index().groupby('card_id').median()[['same_category_3_count']].reset_index()
t5 = t5.rename(columns={'same_category_3_count': 'same_category_3_median'})

t6 = t[['card_id', 'category_3']].groupby('card_id').mean().reset_index()
t6 = t6.rename(columns={'category_3': 'mean_category_3_value'})
t7 = t[['card_id', 'category_3']].groupby('card_id').max().reset_index()
t7 = t7.rename(columns={'category_3': 'max_category_3_value'})
t8 = t[['card_id', 'category_3']].groupby('card_id').min().reset_index()
t8 = t8.rename(columns={'category_3': 'min_category_3_value'})
t9 = t[['card_id', 'category_3']].groupby('card_id').median().reset_index()
t9 = t9.rename(columns={'category_3': 'median_category_3_value'})

In [23]:
category_3_feature = historical[['card_id']]
category_3_feature = category_3_feature.drop_duplicates().reset_index(drop=True)
category_3_feature = pd.merge(category_3_feature, t2, on='card_id', how='left')
category_3_feature = pd.merge(category_3_feature, t3, on='card_id', how='left')
category_3_feature = pd.merge(category_3_feature, t4, on='card_id', how='left')
category_3_feature = pd.merge(category_3_feature, t5, on='card_id', how='left')
category_3_feature = pd.merge(category_3_feature, t6, on='card_id', how='left')
category_3_feature = pd.merge(category_3_feature, t7, on='card_id', how='left')
category_3_feature = pd.merge(category_3_feature, t8, on='card_id', how='left')
category_3_feature = pd.merge(category_3_feature, t9, on='card_id', how='left')

In [24]:
category_3_feature.to_csv('data/category_3_feature.csv', index=None)

## authorized特征——值得深入

In [32]:
historical['authorized_flag'] = historical['authorized_flag'].map(dict(zip(historical['authorized_flag'].unique(), range(0, historical['authorized_flag'].nunique()))))

In [33]:
authorized_feature = historical[['card_id']]
authorized_feature = authorized_feature.drop_duplicates().reset_index(drop=True)
t = historical[['merchant_id','authorized_flag', 'card_id']]
t['same_authorized_count'] = 1
t1 = t[['card_id', 'authorized_flag', 'same_authorized_count']].groupby(['card_id', 'authorized_flag']).sum()

t2 = t1.reset_index().groupby('card_id').max()[['same_authorized_count']].reset_index()
t2 = t2.rename(columns={'same_authorized_count': 'same_authorized_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['same_authorized_count']].reset_index()
t3 = t3.rename(columns={'same_authorized_count': 'same_authorized_mean'})
t4 = t1.reset_index().groupby('card_id').min()[['same_authorized_count']].reset_index()
t4 = t4.rename(columns={'same_authorized_count': 'same_authorized_min'})
t5 = t1.reset_index().groupby('card_id').median()[['same_authorized_count']].reset_index()
t5 = t5.rename(columns={'same_authorized_count': 'same_authorized_median'})

t6 = t[['card_id', 'authorized_flag']].groupby('card_id').mean().reset_index()
t6 = t6.rename(columns={'authorized_flag': 'mean_authorized_value'})
t7 = t[['card_id', 'authorized_flag']].groupby('card_id').max().reset_index()
t7 = t7.rename(columns={'authorized_flag': 'max_authorized_value'})
t8 = t[['card_id', 'authorized_flag']].groupby('card_id').median().reset_index()
t8 = t8.rename(columns={'authorized_flag': 'median_authorized_value'})

In [34]:
authorized_feature = pd.merge(authorized_feature, t2, on='card_id', how='left')
authorized_feature = pd.merge(authorized_feature, t3, on='card_id', how='left')
authorized_feature = pd.merge(authorized_feature, t4, on='card_id', how='left')
authorized_feature = pd.merge(authorized_feature, t5, on='card_id', how='left')
authorized_feature = pd.merge(authorized_feature, t6, on='card_id', how='left')
authorized_feature = pd.merge(authorized_feature, t7, on='card_id', how='left')
authorized_feature = pd.merge(authorized_feature, t8, on='card_id', how='left')

In [42]:
authorized_feature.to_csv('data/authorized_feature.csv', index=None)

## Month_lag特征——值得深入

In [6]:
month_lag_feature = historical[['card_id']]
month_lag_feature = month_lag_feature.drop_duplicates().reset_index(drop=True)
month_lag_feature['purchase_month'] = historical['purchase_day']//30 + historical['month_lag']
t = historical[['merchant_id','month_lag', 'card_id']]
t['same_month_lag_count'] = 1
t1 = t[['card_id', 'month_lag', 'same_month_lag_count']].groupby(['card_id', 'month_lag']).count()

t2 = t1.reset_index().groupby('card_id').max()[['same_month_lag_count']].reset_index()
t2 = t2.rename(columns={'same_month_lag_count': 'same_month_lag_max'})
t3 = t1.reset_index().groupby('card_id').mean()[['same_month_lag_count']].reset_index()
t3 = t3.rename(columns={'same_month_lag_count': 'same_month_lag_mean'})
t4 = t1.reset_index().groupby('card_id').min()[['same_month_lag_count']].reset_index()
t4 = t4.rename(columns={'same_month_lag_count': 'same_month_lag_min'})
t5 = t1.reset_index().groupby('card_id').median()[['same_month_lag_count']].reset_index()
t5 = t5.rename(columns={'same_month_lag_count': 'same_month_lag_median'})

t6 = t[['card_id', 'month_lag']].groupby('card_id').mean().reset_index()
t6 = t6.rename(columns={'month_lag': 'mean_month_lag_value'})
t7 = t[['card_id', 'month_lag']].groupby('card_id').max().reset_index()
t7 = t7.rename(columns={'month_lag': 'max_month_lag_value'})
t8 = t[['card_id', 'month_lag']].groupby('card_id').min().reset_index()
t8 = t8.rename(columns={'month_lag': 'min_month_lag_value'})
t9 = t[['card_id', 'month_lag']].groupby('card_id').median().reset_index()
t9 = t9.rename(columns={'month_lag': 'median_month_lag_value'})

In [7]:
month_lag_feature = pd.merge(month_lag_feature, t2, on='card_id', how='left')
month_lag_feature = pd.merge(month_lag_feature, t3, on='card_id', how='left')
month_lag_feature = pd.merge(month_lag_feature, t4, on='card_id', how='left')
month_lag_feature = pd.merge(month_lag_feature, t5, on='card_id', how='left')
month_lag_feature = pd.merge(month_lag_feature, t6, on='card_id', how='left')
month_lag_feature = pd.merge(month_lag_feature, t7, on='card_id', how='left')
month_lag_feature = pd.merge(month_lag_feature, t8, on='card_id', how='left')
month_lag_feature = pd.merge(month_lag_feature, t9, on='card_id', how='left')

In [8]:
month_lag_feature.head()

,card_id,purchase_month,same_month_lag_max,same_month_lag_mean,same_month_lag_min,same_month_lag_median,mean_month_lag_value,max_month_lag_value,min_month_lag_value,median_month_lag_value
0,C_ID_4e6213e9bc,59,64,28.571429,2,29.5,-6.072500,0,-13,-6.0
1,C_ID_5037ff576e,61,16,9.833333,4,10.0,-5.881356,0,-11,-5.0
2,C_ID_0e171c1b48,63,30,21.461538,4,22.0,-5.555556,0,-12,-6.0
3,C_ID_48fb13e70f,65,28,8.888889,1,6.0,-4.625000,0,-8,-4.0
4,C_ID_fc8e41b9cf,53,37,30.250000,22,30.0,-3.355372,0,-7,-3.0


In [9]:
month_lag_feature.to_csv('data/month_lag_feature.csv', index=None)